<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-10-08 16:44:20


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    70
1    66
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    28
V40     24
VR       7
AR       4
V200     1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    125
0     11
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    27
0    25
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 30, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14
Date,,,,,,,,
2023-09-06 00:00:00+05:30,19611.0,19488.0,0.63,^NSEI,20192.35,16945.05,15.73,59.0
2023-09-13 00:00:00+05:30,20070.0,19522.0,2.81,^NSEI,20192.35,16945.05,18.44,73.0
2023-09-21 00:00:00+05:30,19742.0,19605.0,0.70,^NSEI,20192.35,16945.05,16.51,51.0
2023-09-28 00:00:00+05:30,19524.0,19630.0,-0.54,^NSEI,20192.35,16945.05,15.22,42.0
2023-10-06 00:00:00+05:30,19654.0,19660.0,-0.03,^NSEI,20192.35,16945.05,15.98,50.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Min', 'Close'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
BLUESTARCO.NS,7.31,54.87,60.0,58.3,24.4,23.40,AR,BTT,1,1,1.0,1.0,1.0,H,AC,73.0
PNB.NS,6.60,117.73,56.0,19.2,4.1,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,23.0
PNB.NS,6.60,117.73,56.0,19.2,4.1,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,23.0
MOTILALOFS.NS,4.24,67.72,60.0,9.6,12.9,15.60,V40N,BTT,1,1,0.0,NaN,0.0,L,FINANCE,18.0
AKZOINDIA.NS,-4.50,25.83,37.0,31.0,34.7,26.10,V40,BTT,1,1,1.0,1.0,NaN,M,NaN,16.0
BAJAJHIND.NS,-1.15,153.20,52.0,1000.0,1000.0,1000.00,VR,BTT,1,0,1.0,1.0,1.0,M,SUGAR,12.0
BAJAJHIND.NS,-1.15,153.20,52.0,1000.0,1000.0,1000.00,VR,BTT,1,0,1.0,1.0,1.0,M,SUGAR,12.0
ASIANPAINT.NS,-1.34,18.90,44.0,65.8,34.4,27.70,V40,BTT,1,1,1.0,1.0,1.0,H,PAINTS,11.0
ABBOTINDIA.NS,0.28,28.98,51.0,47.3,41.1,31.60,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,7.0


In [15]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
KANSAINER.NS,-1.58,30.83,47.0,48.4,14.5,10.7,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,35.0
VINATIORGA.NS,-0.34,8.75,50.0,43.4,30.4,22.6,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,25.0
MOTILALOFS.NS,4.24,67.72,60.0,9.6,12.9,15.6,V40N,BTT,1,1,0.0,NaN,0.0,L,FINANCE,18.0
CERA.NS,-7.63,62.60,34.0,46.2,25.5,19.4,V40N,NaN,1,1,1.0,1.0,1.0,H,CERAMICS,17.0
DABUR.NS,-2.12,8.86,39.0,56.5,22.7,19.5,V40,NaN,1,1,0.0,1.0,1.0,H,FMCG,17.0
AKZOINDIA.NS,-4.50,25.83,37.0,31.0,34.7,26.1,V40,BTT,1,1,1.0,1.0,NaN,M,NaN,16.0
DIXON.NS,1.54,94.08,55.0,110.0,24.2,22.4,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,12.0
ITC.NS,-0.64,38.49,47.0,27.8,39.0,29.1,V40,NaN,1,1,0.0,NaN,1.0,H,FMCG,11.0
ASIANPAINT.NS,-1.34,18.90,44.0,65.8,34.4,27.7,V40,BTT,1,1,1.0,1.0,1.0,H,PAINTS,11.0


In [16]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ADANIGREEN.NS,-2.81,107.97,39.0,124.0,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,209.0
BLUESTARCO.NS,7.31,54.87,60.0,58.3,24.40,23.40,AR,BTT,1,1,1.0,1.0,1.0,H,AC,73.0
MUTHOOTFIN.NS,-5.12,34.19,38.0,12.7,12.10,17.90,NaN,NaN,1,1,0.0,NaN,0.0,L,FINANCE,37.0
KANSAINER.NS,-1.58,30.83,47.0,48.4,14.50,10.70,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,35.0
BPCL.NS,-1.08,20.17,46.0,3.7,6.86,6.34,NaN,NaN,1,1,0.0,1.0,NaN,L,NaN,26.0
VINATIORGA.NS,-0.34,8.75,50.0,43.4,30.40,22.60,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,25.0
TASTYBITE.NS,0.99,107.16,56.0,1000.0,1000.00,1000.00,NaN,NaN,1,1,1.0,NaN,NaN,M,NaN,23.0
PNB.NS,6.60,117.73,56.0,19.2,4.10,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,23.0
PNB.NS,6.60,117.73,56.0,19.2,4.10,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,23.0


In [17]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
JCHAC.NS,-3.77,27.95,47.0,1000.0,1000.00,1000.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,167.0
TANLA.NS,0.42,101.15,46.0,28.4,37.90,31.20,AR,ATH,0,0,1.0,1.0,0.0,M,IT,99.0
SFL.NS,-3.66,14.02,39.0,56.2,13.60,12.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,82.0
BLUESTARCO.NS,7.31,54.87,60.0,58.3,24.40,23.40,AR,BTT,1,1,1.0,1.0,1.0,H,AC,73.0
LALPATHLAB.NS,7.34,40.03,62.0,78.5,18.20,15.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,68.0
SYMPHONY.NS,-1.80,6.19,41.0,54.2,14.80,13.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,68.0
WHIRLPOOL.NS,-0.05,32.02,55.0,99.6,8.43,6.37,V40,NaN,1,0,0.0,NaN,1.0,M,NaN,61.0
DREAMFOLKS.NS,3.91,45.05,52.0,37.2,76.80,60.20,AR,ATH,0,0,0.0,1.0,0.0,L,MISC,59.0
PFIZER.NS,3.21,16.02,68.0,119.0,27.20,19.80,V40,NaN,1,0,0.0,NaN,0.0,L,PHARMA,50.0


In [18]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
YESBANK.NS,-3.89,17.29,47.0,64.8,4.94,1.99,NaN,NaN,1,0,0.0,NaN,0.0,L,NaN,1505.0
ADANIGREEN.NS,-2.81,107.97,39.0,124.0,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,209.0
JCHAC.NS,-3.77,27.95,47.0,1000.0,1000.00,1000.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,167.0
ALKYLAMINE.NS,-3.78,7.67,41.0,62.2,41.30,41.40,NaN,NaN,0,0,1.0,1.0,0.0,M,CHEMICALS,99.0
ZEEL.NS,-2.38,50.40,47.0,120.0,7.96,3.30,NaN,NaN,1,0,0.0,NaN,0.0,L,ENTERTAINMENT,85.0
SFL.NS,-3.66,14.02,39.0,56.2,13.60,12.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,82.0
BIOCON.NS,-0.55,34.70,48.0,44.3,5.92,4.78,NaN,NaN,1,0,0.0,1.0,1.0,M,NaN,80.0
ADANIENT.NS,-0.40,107.84,50.0,102.0,9.49,9.63,NaN,NaN,1,0,0.0,1.0,1.0,M,NaN,68.0
SYMPHONY.NS,-1.80,6.19,41.0,54.2,14.80,13.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,68.0


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 11


,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ASTRAZEN.NS,7.19,60.18,70.0,71.7,30.90,23.0,V40N,NaN,1,1,1.0,1.0,NaN,M,NaN,0.0
BAJFINANCE.NS,8.60,48.86,74.0,40.1,11.80,23.5,V40,BTT,1,1,1.0,1.0,1.0,H,FINANCE,-0.0
UJJIVANSFB.NS,17.84,165.23,75.0,1000.0,1000.00,1000.0,V40N,BTT,1,1,1.0,1.0,1.0,M,BANKS,0.0
ANGELONE.NS,6.96,100.53,62.0,17.8,44.00,47.1,V40N,NaN,1,1,1.0,1.0,NaN,M,FINANCE,1.0
TITAN.NS,3.75,44.35,63.0,91.3,25.10,30.8,V40,NaN,1,1,0.0,NaN,1.0,H,JEWELLERY,1.0
POLYCAB.NS,1.47,110.39,55.0,54.4,27.00,20.0,V40N,NaN,1,1,1.0,NaN,1.0,M,CABLES,2.0
BAJAJ-AUTO.NS,3.10,51.13,57.0,21.7,26.20,20.2,V40,NaN,1,1,0.0,1.0,NaN,L,NaN,3.0
CAPLIPOINT.NS,3.64,85.80,62.0,20.8,26.30,22.3,V40N,NaN,1,1,1.0,1.0,NaN,M,NaN,3.0
AXISBANK.NS,-0.20,29.11,48.0,13.5,6.15,14.5,V40,NaN,1,1,0.0,1.0,1.0,M,BANKS,4.0


In [20]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
3MINDIA.NS,-0.49,45.88,52.0,70.80,31.40,23.40,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,6.0
ABBOTINDIA.NS,0.28,28.98,51.0,47.30,41.10,31.60,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,7.0
ACC.NS,-0.20,25.81,51.0,44.40,10.00,7.06,NaN,NaN,0,0,0.0,1.0,0.0,L,CEMENT,35.0
ADANIENT.NS,-0.40,107.84,50.0,102.00,9.49,9.63,NaN,NaN,1,0,0.0,1.0,1.0,M,NaN,68.0
ADANIGREEN.NS,-2.81,107.97,39.0,124.00,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,209.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VOLTAS.NS,1.62,20.08,59.0,88.60,9.67,4.42,NaN,NaN,1,0,0.0,NaN,0.0,L,AC,49.0
WHIRLPOOL.NS,-0.05,32.02,55.0,99.60,8.43,6.37,V40,NaN,1,0,0.0,NaN,1.0,M,NaN,61.0
WSTCSTPAPR.NS,4.94,55.41,63.0,4.63,52.90,44.80,VR,ATH,0,0,0.0,NaN,NaN,L,PAPER,4.0


In [21]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [22]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14
Date,,,,,,,,
2023-09-06 00:00:00+05:30,596.0,585.0,1.96,BERGEPAINT.NS,710.11,447.11,33.40,59.0
2023-09-13 00:00:00+05:30,603.0,591.0,2.09,BERGEPAINT.NS,710.11,447.11,34.94,62.0
2023-09-21 00:00:00+05:30,628.0,595.0,5.50,BERGEPAINT.NS,710.11,447.11,40.39,71.0
2023-09-28 00:00:00+05:30,570.0,600.0,-5.08,BERGEPAINT.NS,710.11,447.11,27.38,40.0
2023-10-06 00:00:00+05:30,565.0,596.0,-5.14,BERGEPAINT.NS,710.11,447.11,26.45,40.0
